In [12]:
from collections import defaultdict
import json
import numpy as np
import pandas as pd
import requests
import random

In [15]:
# placeholder for making API calls

KEY = 'your_key'
CURSOR = 0
f"""
curl -X 'GET' \
  'https://vote.optimism.io/api/v1/retrofunding/rounds/5/projects?limit=100&offset={CURSOR}&category=all' \
  -H 'accept: application/json' \
  -H 'Authorization: Bearer {KEY}'
"""

print("Fetch the data...")

Fetch the data...


In [2]:
# Simulation constants
NUM_CATEGORIES = 5
NUM_PROJECTS = 200
NUM_VOTERS = 100
MAX_OP = 1_000_000
MIN_OP = 1000
TOTAL_OP = 8_000_000
MAX_COI_PROJECTS = 10

In [3]:
# Initialize categories and projects
categories = [f"Category_{i}" for i in range(NUM_CATEGORIES)]
projects = [f"Project_{i}" for i in range(NUM_PROJECTS)]

# Randomly allocate projects to categories
projects_by_category = {category: [] for category in categories}
for project in projects:
    category = random.choice(categories)
    projects_by_category[category].append(project)

In [4]:
projects_by_category

{'Category_0': ['Project_0',
  'Project_5',
  'Project_19',
  'Project_25',
  'Project_29',
  'Project_34',
  'Project_39',
  'Project_43',
  'Project_53',
  'Project_68',
  'Project_74',
  'Project_77',
  'Project_80',
  'Project_88',
  'Project_91',
  'Project_94',
  'Project_96',
  'Project_100',
  'Project_107',
  'Project_109',
  'Project_112',
  'Project_120',
  'Project_121',
  'Project_128',
  'Project_130',
  'Project_131',
  'Project_143',
  'Project_145',
  'Project_150',
  'Project_153',
  'Project_165',
  'Project_167',
  'Project_172',
  'Project_174',
  'Project_181',
  'Project_184',
  'Project_189'],
 'Category_1': ['Project_1',
  'Project_2',
  'Project_10',
  'Project_23',
  'Project_28',
  'Project_30',
  'Project_31',
  'Project_32',
  'Project_33',
  'Project_38',
  'Project_40',
  'Project_41',
  'Project_48',
  'Project_49',
  'Project_54',
  'Project_62',
  'Project_79',
  'Project_82',
  'Project_85',
  'Project_87',
  'Project_93',
  'Project_101',
  'Project

In [5]:
# Generate voter ballots
def generate_ballot():

    # Generate random category for the voter
    assigned_category = random.sample(categories, 1)[0]

    # Generate random COI list for each voter (for simplicity, random projects)
    coi_projects = list(random.sample(projects, random.randint(0, MAX_COI_PROJECTS)))
    
    #  Determine category allocations
    category_allocations = {category: random.randint(0, 10000)/100 for category in categories}
    total = sum(category_allocations.values())
    category_allocations = {k: (v * 100 / total) for k, v in category_allocations.items()}

    # Determine project allocations within a category
    for category, proj_list in projects_by_category.items():
        if category != assigned_category:
            continue
        allocation = {project: random.randint(0, 10000)/100 for project in proj_list}
        total = sum(allocation.values())
        allocation = {k: (v * 100 / total) for k, v in allocation.items()}

    return {
        "assigned_category": assigned_category,
        "conflicts_of_interest": coi_projects,
        "category_allocations": category_allocations,
        "project_allocations": allocation
    }

# Null any projects with a COI
def process_ballot(ballot):
    cois = ballot['conflicts_of_interest']
    allocation = ballot['project_allocations'].copy()
    allocation = {k: v for k, v in allocation.items() if k not in cois}
    total = sum(allocation.values())
    allocation = {k: (v * 100 / total) for k, v in allocation.items()}
    ballot.update({'cleaned_project_allocations': allocation})
    return ballot

# Generate all voter ballots
voter_ballots = [generate_ballot() for _ in range(NUM_VOTERS)]

# Create JSON dump of ballot data for testing
# with open("rf5_dummy_ballots.json", "w") as f:
#     json.dump(voter_ballots, f, indent=2)

# Process all ballots
voter_ballots = [process_ballot(ballot) for ballot in voter_ballots]

In [6]:
voter_ballots[2]

{'assigned_category': 'Category_2',
 'conflicts_of_interest': ['Project_174',
  'Project_198',
  'Project_109',
  'Project_114',
  'Project_33',
  'Project_103',
  'Project_173',
  'Project_144'],
 'category_allocations': {'Category_0': 51.78183136023194,
  'Category_1': 13.868084078279777,
  'Category_2': 14.411693645808166,
  'Category_3': 3.158975597970524,
  'Category_4': 16.779415317709592},
 'project_allocations': {'Project_3': 2.9069644525815965,
  'Project_6': 0.22071788279964996,
  'Project_12': 4.186874474103705,
  'Project_13': 4.071018727193544,
  'Project_14': 2.838465799298947,
  'Project_16': 2.9496704030849776,
  'Project_18': 2.230857374810255,
  'Project_20': 1.1712424048946943,
  'Project_21': 1.183081678301572,
  'Project_27': 3.177576416167375,
  'Project_42': 3.024088693071066,
  'Project_51': 2.048194299389855,
  'Project_57': 2.0249385837692024,
  'Project_58': 1.7128891632593524,
  'Project_59': 1.0570779827569443,
  'Project_64': 1.479486344666619,
  'Project_

In [7]:
# Process votes and calculate scores
def calculate_scores(voter_ballots):
    category_scores = defaultdict(list)
    project_scores = defaultdict(lambda: defaultdict(list))

    for ballot in voter_ballots:
        assigned_category = ballot['assigned_category']
        for category, percentage in ballot['category_allocations'].items():
            category_scores[category].append(percentage)

        for project, percentage in ballot['cleaned_project_allocations'].items():
            project_scores[assigned_category][project].append(percentage)

    # Calculate medians for categories
    final_category_scores = {k: np.median(v) for k, v in category_scores.items()}
    
    # Normalize to 100%
    total = sum(final_category_scores.values())
    final_category_scores = {k: v * 100 / total for k, v in final_category_scores.items()}
    
    # Calculate medians for projects and normalize
    final_project_scores = {}
    for category, projects in project_scores.items():
        category_total = sum(np.median(v) for v in projects.values())
        final_project_scores[category] = {k: np.median(v) * 100 / category_total for k, v in projects.items()}
    
    return final_category_scores, final_project_scores

# Calculate final results
def calculate_final_results(category_scores, project_scores):
    final_results = {}
    for category, projects in project_scores.items():
        category_score = category_scores[category]
        final_results[category] = {
            project: TOTAL_OP * (category_score/100) * (project_score / 100)
            for project, project_score in projects.items()
        }

    # Adjust for max and min OP constraints
    for category, projects in final_results.items():
        for project, score in projects.items():
            if score > MAX_OP:
                excess = score - MAX_OP
                score = MAX_OP
                # Redistribute excess (simplified)
            if score < MIN_OP:
                deficit = MIN_OP - score
                score = MIN_OP
                # Redistribute deficit (simplified)
            final_results[category][project] = score

    return final_results


category_scores, project_scores = calculate_scores(voter_ballots)
print("Final Category Scores:")
pd.Series(category_scores)

Final Category Scores:


Category_0    19.006806
Category_1    21.666615
Category_2    17.887071
Category_3    21.690650
Category_4    19.748856
dtype: float64

In [8]:
print("Final Project Percentages:")
pd.DataFrame(project_scores).fillna(0).sum(axis=1).sort_values()

Final Project Percentages:


Project_23     1.211952
Project_126    1.233213
Project_190    1.333434
Project_118    1.378242
Project_132    1.422221
                 ...   
Project_8      3.673665
Project_188    3.781016
Project_80     3.783185
Project_26     3.852254
Project_181    3.947574
Length: 200, dtype: float64

In [9]:
# Calculate final results
def calculate_final_results(category_scores, project_scores):
    final_results = {}
    for category, projects in project_scores.items():
        category_score = category_scores[category]
        final_results[category] = {
            project: TOTAL_OP * (category_score/100) * (project_score / 100)
            for project, project_score in projects.items()
        }

    # Adjust for max and min OP constraints
    for category, projects in final_results.items():
        for project, score in projects.items():
            if score > MAX_OP:
                excess = score - MAX_OP
                score = MAX_OP
                # Redistribute excess (simplified)
            if score < MIN_OP:
                deficit = MIN_OP - score
                score = MIN_OP
                # Redistribute deficit (simplified)
            final_results[category][project] = score

    return final_results

final_results = calculate_final_results(category_scores, project_scores)

print("Final Results:")
pd.DataFrame(final_results).fillna(0).sum(axis=1).sort_values()

Final Results:


Project_132    20351.498823
Project_23     21007.114182
Project_190    21067.035496
Project_126    21399.361950
Project_117    21557.677525
                   ...     
Project_180    58072.300322
Project_181    60024.613215
Project_26     60862.087403
Project_11     62600.912551
Project_188    65610.161976
Length: 200, dtype: float64

# Test dummy ballots

In [10]:
dummy_data = json.load(open("rf5_dummy_ballots.json"))

voter_ballots = [process_ballot(ballot) for ballot in dummy_data]
category_scores, project_scores = calculate_scores(voter_ballots)
final_results = calculate_final_results(category_scores, project_scores)
pd.DataFrame(final_results).fillna(0).sum(axis=1).sort_values()

Project_132    20351.498823
Project_23     21007.114182
Project_190    21067.035496
Project_126    21399.361950
Project_117    21557.677525
                   ...     
Project_180    58072.300322
Project_181    60024.613215
Project_26     60862.087403
Project_11     62600.912551
Project_188    65610.161976
Length: 200, dtype: float64